In [13]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [14]:
#图像处理
transform = transforms.Compose([
    transforms.Scale(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

In [15]:
#CIRAR-10 数据集
train_dataset = dsets.CIFAR10(root='../data/', train=True, transform=transform, download=True)
test_dataset = dsets.CIFAR10(root='../data/', train=True, transform=transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
#导入数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [31]:
#3*3卷积核
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [32]:
#残差块定义
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [33]:
#定义残差网络
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
resnet = ResNet(ResidualBlock, [3, 3, 3])
resnet.cuda()

ResNet (
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (layer1): Sequential (
    (0): ResidualBlock (
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): ResidualBlock (
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    )
    (2): ResidualBlock (
      (

In [34]:
#定义损失函数和优化函数
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)

In [36]:
#训练网络
for epoch in range(80):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        #前向+后向+优化
        optimizer.zero_grad()
        outputs = resnet(images)
        
        #调用损失函数，求输出值与真实值之间的差
        loss = criterion(outputs, labels)
        
        #后向传播
        loss.backward()
        
        #执行优化
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [%d/%d], Iter [%d/%d] Loss: %.4f" %(epoch+1, 80, i+1, 500, loss.data[0]))
            
    #改变学习率
    if (epoch+1) % 20 == 0:
        lr /= 3
        optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)

Epoch [1/80], Iter [100/500] Loss: 1.6360
Epoch [1/80], Iter [200/500] Loss: 1.4460
Epoch [1/80], Iter [300/500] Loss: 1.3988
Epoch [1/80], Iter [400/500] Loss: 1.2689
Epoch [1/80], Iter [500/500] Loss: 1.1688
Epoch [2/80], Iter [100/500] Loss: 1.1628
Epoch [2/80], Iter [200/500] Loss: 1.0404
Epoch [2/80], Iter [300/500] Loss: 0.9676
Epoch [2/80], Iter [400/500] Loss: 0.9105
Epoch [2/80], Iter [500/500] Loss: 0.9309
Epoch [3/80], Iter [100/500] Loss: 1.0631
Epoch [3/80], Iter [200/500] Loss: 0.7920
Epoch [3/80], Iter [300/500] Loss: 0.8698
Epoch [3/80], Iter [400/500] Loss: 0.8733
Epoch [3/80], Iter [500/500] Loss: 1.0725
Epoch [4/80], Iter [100/500] Loss: 0.8598
Epoch [4/80], Iter [200/500] Loss: 0.7920
Epoch [4/80], Iter [300/500] Loss: 0.7636
Epoch [4/80], Iter [400/500] Loss: 0.6817
Epoch [4/80], Iter [500/500] Loss: 0.8084
Epoch [5/80], Iter [100/500] Loss: 0.7398
Epoch [5/80], Iter [200/500] Loss: 0.7437
Epoch [5/80], Iter [300/500] Loss: 0.5037
Epoch [5/80], Iter [400/500] Loss:

Epoch [39/80], Iter [300/500] Loss: 0.2469
Epoch [39/80], Iter [400/500] Loss: 0.1649
Epoch [39/80], Iter [500/500] Loss: 0.2946
Epoch [40/80], Iter [100/500] Loss: 0.2628
Epoch [40/80], Iter [200/500] Loss: 0.3010
Epoch [40/80], Iter [300/500] Loss: 0.2301
Epoch [40/80], Iter [400/500] Loss: 0.3662
Epoch [40/80], Iter [500/500] Loss: 0.1662
Epoch [41/80], Iter [100/500] Loss: 0.1998
Epoch [41/80], Iter [200/500] Loss: 0.1358
Epoch [41/80], Iter [300/500] Loss: 0.2880
Epoch [41/80], Iter [400/500] Loss: 0.2664
Epoch [41/80], Iter [500/500] Loss: 0.2238
Epoch [42/80], Iter [100/500] Loss: 0.2091
Epoch [42/80], Iter [200/500] Loss: 0.2014
Epoch [42/80], Iter [300/500] Loss: 0.1426
Epoch [42/80], Iter [400/500] Loss: 0.1576
Epoch [42/80], Iter [500/500] Loss: 0.2049
Epoch [43/80], Iter [100/500] Loss: 0.2515
Epoch [43/80], Iter [200/500] Loss: 0.1819
Epoch [43/80], Iter [300/500] Loss: 0.2493
Epoch [43/80], Iter [400/500] Loss: 0.2025
Epoch [43/80], Iter [500/500] Loss: 0.3257
Epoch [44/8

Epoch [77/80], Iter [400/500] Loss: 0.1595
Epoch [77/80], Iter [500/500] Loss: 0.0531
Epoch [78/80], Iter [100/500] Loss: 0.1278
Epoch [78/80], Iter [200/500] Loss: 0.0752
Epoch [78/80], Iter [300/500] Loss: 0.1548
Epoch [78/80], Iter [400/500] Loss: 0.1150
Epoch [78/80], Iter [500/500] Loss: 0.1644
Epoch [79/80], Iter [100/500] Loss: 0.1190
Epoch [79/80], Iter [200/500] Loss: 0.1369
Epoch [79/80], Iter [300/500] Loss: 0.1942
Epoch [79/80], Iter [400/500] Loss: 0.1814
Epoch [79/80], Iter [500/500] Loss: 0.1138
Epoch [80/80], Iter [100/500] Loss: 0.0996
Epoch [80/80], Iter [200/500] Loss: 0.1069
Epoch [80/80], Iter [300/500] Loss: 0.1273
Epoch [80/80], Iter [400/500] Loss: 0.1473
Epoch [80/80], Iter [500/500] Loss: 0.0969


In [37]:
#网络测试
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.cuda())
    outputs = resnet(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    
print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))

Accuracy of the model on the test images: 94 %


In [38]:
#模型保存
torch.save(resnet.state_dict(), 'resnet.pkl')